In [1]:
from multiprocessing.dummy import Pool as ThreadPool
import time
try:
    from urllib.request import urlopen
except ImportError:
    from urllib2 import urlopen
import pandas as pd
import json
import os
    
## https://financialmodellingprep.com/developer/docs/company-enterprise-value-api

In [2]:
def get_jsonparsed_data(url):
    """
    Receive the content of \`\`url\`\`, parse it as JSON and return the object.

    Parameters
    ----------
    url : str

    Returns
    -------
    dict
    """
    response = urlopen(url)
    data = response.read().decode("utf-8")
    return json.loads(data)

def get_fin_data(Tickers):
    
    maindf = pd.DataFrame()

    for Ticker in Tickers:
        temp_co = pd.DataFrame()
        
        try:
            
            statements = ['income-statement', 
              'balance-sheet-statement', 
              'cash-flow-statement']

            for statement in statements:

                url = ("https://financialmodelingprep.com/api/v3/financials/" +str(statement) + "/" + str(Ticker))

                a = get_jsonparsed_data(url)

                df = pd.DataFrame.from_records(a.get('financials'))
                try:
                    df.date = df.date.apply(lambda x: str(x).replace('-','')).astype('int32')
                    df = df.loc[df['date']==int(df.date.max())]
                except:
                    pass
                df = df.T
                df[Ticker] = df[0]
                del df[0]
                df = pd.DataFrame(df.loc[df.index != 'date'])
                temp_co = temp_co.append(df)

            url = ("https://financialmodelingprep.com/api/v3/company/profile/" + str(Ticker))
            a = get_jsonparsed_data(url)
            df = pd.DataFrame.from_records(a)
            a = a.get('symbol')
            df[a] = df['profile']
            del df['profile']
            del df['symbol']

            temp_co = temp_co.append(df)

            url = ("https://financialmodelingprep.com/api/v3/enterprise-value/" + str(Ticker))
            a = get_jsonparsed_data(url).get('enterpriseValues')
            df = pd.DataFrame.from_records(a)
            try:
                df.date = df.date.apply(lambda x: str(x).replace('-','')).astype('int32')
                df = df.loc[df['date']==int(df.date.max())]
            except:
                pass
            
            df = df.T
            df[Ticker] = df[0]
            del df[0]
            df = pd.DataFrame(df.loc[df.index != 'date'])
            temp_co = temp_co.append(df)
            
            url = ("https://financialmodelingprep.com/api/v3/company-key-metrics/" + str(Ticker))
            a = get_jsonparsed_data(url).get('metrics')
            df = pd.DataFrame.from_records(a)
            try:
                df.date = df.date.apply(lambda x: str(x).replace('-','')).astype('int32')
                df = df.loc[df['date']==int(df.date.max())]
            except:
                pass
            
            df = df.T
            df[Ticker] = df[0]
            del df[0]
            df = pd.DataFrame(df.loc[df.index != 'date'])
            temp_co = temp_co.append(df)
            maindf = pd.concat([maindf, temp_co], axis=1)
            print(maindf.shape)
        
        except:
            continue
        
    try:
        os.remove(str(Tickers[0])+'_all_fin_data.csv')
    except:
        pass
    
    print(maindf.shape)
    return maindf.to_csv(str(Tickers[0])+'_all_fin_data.csv')



In [3]:
import os

def multi_task_fin():
    Tickers = get_jsonparsed_data('https://financialmodelingprep.com/api/v3/company/stock/list')
    Tickers = pd.DataFrame.from_records(Tickers.get('symbolsList'))
    Tickers = list(Tickers['symbol'].values)
    #Tickers = Tickers[:10]
    tn = int(len(Tickers)/4)
    t1 = Tickers[:tn]
    t2 = Tickers[tn:tn*2]
    t3 = Tickers[tn*2:tn*3]
    t4 = Tickers[tn*3:]

    tl = [t1, t2, t3, t4]
    
    # make the Pool of workers
    pool = ThreadPool(8)
    results = pool.map(get_fin_data, tl)
    
    # close the pool and wait for the work to finish
    pool.close()
    pool.join()
    
    import os

    directory = str(os.getcwd())
    old_snap = []
    for file in os.listdir(directory):
        if file.endswith('_all_fin_data.csv'):
            old_snap.append(file)

    big = pd.DataFrame()
    
    for file in old_snap:
        t = pd.read_csv(file)
        t['Ticker'] = t['Unnamed: 0']
        del t['Unnamed: 0']
        t = t.set_index('Ticker')
        big = pd.concat([big, t], axis=1)
    
    print("Done!")
    
    try:
        for file in old_snap():
            os.remove(file)
    except:
        pass
    
    return big

In [4]:
df = multi_task_fin()
df.sample(n=20)

(154, 1)
(154, 1)
(154, 1)
(154, 1)
(154, 2)
(154, 2)
(154, 2)
(154, 2)
(154, 3)
(154, 3)
(154, 3)
(154, 3)
(154, 4)
(154, 4)
(154, 4)
(154, 5)
(154, 5)
(154, 5)
(154, 4)
(154, 6)
(154, 6)
(154, 6)
(154, 5)
(154, 7)
(154, 7)
(154, 7)
(154, 6)
(154, 8)
(154, 8)
(154, 8)
(154, 7)
(154, 9)
(154, 9)
(154, 10)
(154, 8)
(154, 10)
(154, 9)
(154, 11)
(154, 9)
(154, 11)
(154, 10)
(154, 12)
(154, 10)
(154, 12)
(154, 11)
(154, 13)
(154, 11)
(154, 13)
(154, 14)
(154, 12)
(154, 14)
(154, 12)
(154, 15)
(154, 13)
(154, 15)
(154, 13)
(154, 16)
(154, 14)
(154, 16)
(154, 17)
(154, 14)
(154, 15)
(154, 17)
(154, 18)
(154, 16)
(154, 18)
(154, 19)
(154, 17)
(154, 19)
(154, 20)
(154, 18)
(154, 20)
(154, 21)
(154, 19)
(154, 21)
(154, 22)
(154, 15)
(154, 20)
(154, 22)
(154, 23)
(154, 23)
(154, 21)
(154, 24)
(154, 24)
(154, 22)
(154, 16)
(154, 25)
(154, 25)
(154, 23)
(154, 17)
(154, 26)
(154, 26)
(154, 24)
(154, 27)
(154, 27)
(154, 25)
(154, 18)
(154, 28)
(154, 26)
(154, 28)
(154, 29)
(154, 29)
(154, 27)
(154, 

(154, 224)
(154, 227)
(154, 209)
(154, 129)
(154, 225)
(154, 228)
(154, 210)
(154, 226)
(154, 229)
(154, 130)
(154, 211)
(154, 227)
(154, 230)
(154, 131)
(154, 212)
(154, 228)
(154, 231)
(154, 132)
(154, 229)
(154, 232)
(154, 213)
(154, 230)
(154, 233)
(154, 214)
(154, 231)
(154, 133)
(154, 234)
(154, 215)
(154, 232)
(154, 134)
(154, 233)
(154, 235)
(154, 216)
(154, 234)
(154, 236)
(154, 235)
(154, 237)
(154, 236)
(154, 135)
(154, 238)
(154, 217)
(154, 237)
(154, 239)
(154, 238)
(154, 240)
(154, 218)
(154, 239)
(154, 241)
(154, 240)
(154, 219)
(154, 136)
(154, 242)
(154, 241)
(154, 220)
(154, 137)
(154, 243)
(154, 242)
(154, 221)
(154, 138)
(154, 244)
(154, 243)
(154, 222)
(154, 139)
(154, 245)
(154, 244)
(154, 223)
(154, 140)
(154, 246)
(154, 224)
(154, 247)
(154, 245)
(154, 225)
(154, 248)
(154, 246)
(154, 141)
(154, 226)
(154, 249)
(154, 247)
(154, 227)
(154, 250)
(154, 248)
(154, 228)
(154, 251)
(154, 142)
(154, 249)
(154, 229)
(154, 252)
(154, 250)
(154, 143)
(154, 253)
(154, 251)

(154, 387)
(154, 449)
(154, 439)
(154, 259)
(154, 388)
(154, 450)
(154, 440)
(154, 260)
(154, 451)
(154, 389)
(154, 441)
(154, 261)
(154, 452)
(154, 442)
(154, 262)
(154, 453)
(154, 263)
(154, 443)
(154, 390)
(154, 454)
(154, 264)
(154, 444)
(154, 391)
(154, 265)
(154, 445)
(154, 455)
(154, 392)
(154, 456)
(154, 446)
(154, 457)
(154, 266)
(154, 447)
(154, 458)
(154, 393)
(154, 267)
(154, 448)
(154, 459)
(154, 394)
(154, 268)
(154, 460)
(154, 395)
(154, 449)
(154, 269)
(154, 461)
(154, 396)
(154, 450)
(154, 462)
(154, 397)
(154, 270)
(154, 463)
(154, 271)
(154, 398)
(154, 451)
(154, 464)
(154, 399)
(154, 452)
(154, 465)
(154, 272)
(154, 400)
(154, 453)
(154, 466)
(154, 273)
(154, 401)
(154, 454)
(154, 467)
(154, 274)
(154, 455)
(154, 468)
(154, 402)
(154, 469)
(154, 456)
(154, 403)
(154, 470)
(154, 404)
(154, 457)
(154, 275)
(154, 471)
(154, 405)
(154, 458)
(154, 472)
(154, 406)
(154, 459)
(154, 276)
(154, 473)
(154, 460)
(154, 407)
(154, 277)
(154, 461)
(154, 474)
(154, 278)
(154, 462)

(154, 654)
(154, 665)
(154, 560)
(154, 655)
(154, 400)
(154, 666)
(154, 656)
(154, 561)
(154, 401)
(154, 657)(154, 667)

(154, 562)
(154, 668)
(154, 658)
(154, 402)
(154, 563)
(154, 659)
(154, 403)
(154, 660)
(154, 669)
(154, 661)
(154, 670)
(154, 404)
(154, 662)
(154, 671)
(154, 405)
(154, 663)
(154, 672)
(154, 564)
(154, 406)
(154, 664)
(154, 673)
(154, 565)
(154, 665)
(154, 407)
(154, 674)
(154, 666)
(154, 566)
(154, 675)
(154, 667)
(154, 567)
(154, 676)
(154, 408)
(154, 677)
(154, 668)
(154, 678)
(154, 669)
(154, 679)
(154, 568)
(154, 670)
(154, 569)
(154, 680)
(154, 409)
(154, 671)
(154, 570)
(154, 681)
(154, 672)
(154, 571)
(154, 682)
(154, 673)
(154, 572)
(154, 683)
(154, 674)
(154, 684)
(154, 675)
(154, 685)
(154, 676)
(154, 573)
(154, 686)
(154, 410)
(154, 677)
(154, 574)
(154, 687)
(154, 678)
(154, 575)
(154, 411)
(154, 688)
(154, 679)
(154, 412)
(154, 689)
(154, 680)
(154, 576)
(154, 690)
(154, 413)
(154, 681)
(154, 577)
(154, 691)
(154, 682)
(154, 692)
(154, 578)
(154, 683)

(154, 904)
(154, 912)
(154, 738)
(154, 905)
(154, 913)
(154, 906)
(154, 914)
(154, 907)
(154, 908)
(154, 915)
(154, 739)
(154, 916)
(154, 909)
(154, 917)
(154, 740)
(154, 910)
(154, 918)
(154, 741)
(154, 911)
(154, 919)
(154, 912)
(154, 742)
(154, 920)
(154, 913)
(154, 921)
(154, 914)
(154, 743)
(154, 922)
(154, 915)
(154, 744)
(154, 923)
(154, 916)
(154, 745)
(154, 924)
(154, 917)
(154, 925)
(154, 746)
(154, 747)
(154, 918)
(154, 926)
(154, 748)
(154, 919)
(154, 927)
(154, 749)
(154, 920)
(154, 928)
(154, 750)
(154, 921)
(154, 922)
(154, 929)
(154, 923)
(154, 930)
(154, 751)
(154, 924)
(154, 752)
(154, 931)
(154, 925)
(154, 753)
(154, 932)
(154, 926)
(154, 754)
(154, 933)
(154, 927)
(154, 934)
(154, 755)
(154, 928)
(154, 756)
(154, 935)
(154, 936)
(154, 757)
(154, 929)
(154, 937)
(154, 930)
(154, 938)
(154, 931)
(154, 758)
(154, 939)
(154, 932)
(154, 759)
(154, 940)
(154, 933)
(154, 760)
(154, 941)
(154, 934)
(154, 942)
(154, 761)
(154, 935)
(154, 943)
(154, 762)
(154, 936)
(154, 944)

(154, 1203)
(154, 1191)
(154, 1204)
(154, 1192)
(154, 1205)
(154, 1193)
(154, 1206)
(154, 1194)
(154, 1195)
(154, 1207)
(154, 1196)
(154, 1208)
(154, 1197)
(154, 1209)
(154, 1198)
(154, 1210)
(154, 1199)
(154, 1200)
(154, 1211)
(154, 1201)
(154, 1212)
(154, 1202)
(154, 1213)
(154, 1203)
(154, 1214)
(154, 1204)
(154, 1215)
(154, 1205)
(154, 1216)
(154, 1206)
(154, 1217)
(154, 1207)
(154, 1218)
(154, 1208)
(154, 1219)
(154, 1209)
(154, 1220)
(154, 1210)
(154, 1221)
(154, 1211)
(154, 1222)
(154, 1212)
(154, 1223)
(154, 1213)
(154, 1214)
(154, 1224)
(154, 1215)
(154, 1225)
(154, 1226)
(154, 1216)
(154, 1227)
(154, 1217)
(154, 1228)
(154, 1218)
(154, 1219)
(154, 1229)
(154, 1230)
(154, 1220)
(154, 1231)
(154, 1221)
(154, 1232)
(154, 1222)
(154, 1233)
(154, 1234)
(154, 1223)
(154, 1235)
(154, 1224)
(154, 1236)
(154, 1225)
(154, 1237)
(154, 1226)
(154, 1238)
(154, 1227)
(154, 1239)
(154, 1228)
(154, 1240)
(154, 1229)
(154, 1241)
(154, 1230)
(154, 1242)
(154, 1231)
(154, 1243)
(154, 1244)
(154

(154, 1580)
(154, 1581)
(154, 1582)
(154, 1583)
(154, 1584)
(154, 1585)
(154, 1586)
(154, 1587)
(154, 1588)
(154, 1589)
(154, 1590)
(154, 1591)
(154, 1592)
(154, 1593)
(154, 1594)
(154, 1595)
(154, 1596)
(154, 1597)
(154, 1598)
(154, 1599)
(154, 1600)
(154, 1601)
(154, 1602)
(154, 1603)
(154, 1604)
(154, 1605)
(154, 1606)
(154, 1607)
(154, 1608)
(154, 1609)
(154, 1610)
(154, 1611)
(154, 1612)
(154, 1613)
(154, 1614)
(154, 1615)
(154, 1616)
(154, 1617)
(154, 1618)
(154, 1619)
(154, 1620)
(154, 1621)
(154, 1622)
(154, 1623)
(154, 1624)
(154, 1625)
(154, 1626)
(154, 1627)
(154, 1628)
(154, 1629)
(154, 1630)
(154, 1631)
(154, 1632)
(154, 1633)
(154, 1634)
(154, 1635)
(154, 1636)
(154, 1637)
(154, 1638)
(154, 1639)
(154, 1640)
(154, 1641)
(154, 1642)
(154, 1643)
(154, 1644)
(154, 1645)
(154, 1646)
(154, 1647)
(154, 1648)
(154, 1649)
(154, 1650)
(154, 1651)
(154, 1652)
(154, 1653)
(154, 1654)
(154, 1654)
Done!


,CMCSA,KMI,INTC,MU,GE,BAC,AAPL,MSFT,SIRI,HPQ,...,WVVI,XBIO,XBIT,XELB,XTLB,YRIV,YTEN,ZKIN,ZOM,ZYME
Ticker,,,,,,,,,,,,,,,,,,,,,
SG&A Expense,64822000000.0,601000000.0,6750000000.0,836000000.0,18111000000.0,53381000000.0,18245000000.0,23098000000.0,1311199000.0,4859000000.0,...,10598784.0,4392375.0,5269000.0,24467000.0,755000.0,3755251.0,5071000.0,7020320.0,4521349.0,29457000.0
Enterprise Value,262839637384.45,67987941254.06,228897520000.0,47202896939.93,135787839462.0,-33341189271.04,1019650744600.0,984238135385.6001,32195813032.07,39020341383.12,...,32821927.81,14784254.76,166141441.76,31399525.68,NaN,800516992.4,8685423.13,NaN,114659072.07,269108183.6
Operating Income,19009000000.0,3794000000.0,23316000000.0,7376000000.0,-14557000000.0,37866000000.0,63930000000.0,42959000000.0,1726948000.0,4064000000.0,...,4182715.0,-7276327.0,-20994000.0,3930000.0,-793000.0,-3755251.0,-9274000.0,9617636.0,-16648628.0,-33127000.0
Price to Sales Ratio,1.6391,2.3999,3.0232,2.1066,0.5206,2.6502,3.8903,8.2618,4.3949,0.6533,...,1.4821,0.0,0.0,0.5826,0.0,0.0,14.9666,0.7619,0.0,8.851
Working Capital,-5755000000.0,-1835000000.0,12161000000.0,10113000000.0,NaN,NaN,57101000000.0,106132000000.0,-2325000000.0,-3744000000.0,...,23239067.0,-405817.0,14072000.0,7759000.0,5906000.0,NaN,4736000.0,25216202.0,1484439.0,174383000.0
Interest Coverage,5.2662,2.1455,-184.0556,55.7344,-3.3036,0.0,0.0,0.0,5.0579,3.8668,...,9.6076,14343.7466,0.0,3.8872,0.0,-0.3406,0.0,7.7919,0.0,-206.1386
Capital Expenditure,-12312000000.0,-2924000000.0,-15181000000.0,-9780000000.0,-3537000000.0,0.0,-10495000000.0,-13925000000.0,-355703000.0,-374000000.0,...,-5261865.0,22500.0,-122000.0,-1476000.0,0.0,9064.0,-42000.0,-467138.0,-618997.0,-803000.0
Payout Ratio,0.358,1.098,0.263,0.0,-0.141,0.205,0.251,0.352,0.173,0.169,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Receivables Turnover,9.4801,9.6022,11.4929,5.3974,3.4583,1.432,5.489,4.494,24.3116,12.275,...,11.223,0.0,0.0,3.6305,0.0,0.0,3.9857,1.5678,0.0,28.7756


In [44]:
df.shape

(154, 4484)

In [8]:
import os
try:
    os.remove('all_company_data_1.csv')
except:
    pass
df.to_csv('all_company_data_1.csv')

In [7]:
df

,CMCSA,KMI,INTC,MU,GE,BAC,AAPL,MSFT,SIRI,HPQ,...,WVVI,XBIO,XBIT,XELB,XTLB,YRIV,YTEN,ZKIN,ZOM,ZYME
Ticker,,,,,,,,,,,,,,,,,,,,,
Consolidated Income,11862000000.0,1919000000.0,21053000000.0,6358000000.0,-22443000000.0,28147000000.0,55256000000.0,39240000000.0,1175893000.0,5327000000.0,...,2858580.0,-7300458.0,-21138000.0,1088000.0,2986000.0,-13716485.0,-9170000.0,7103057.0,-16647687.0,-36556000.0
Cost of Revenue,0.0,7288000000.0,27111000000.0,12704000000.0,95461000000.0,0.0,161782000000.0,42910000000.0,2308606000.0,47803000000.0,...,8298240.0,0.0,0.0,2702000.0,0.0,0.0,0.0,36593792.0,0.0,0.0
Dividend per Share,0.917,0.725,1.2,0.0,0.37,0.54,3.0,1.8,0.045,0.557,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
EBIT,18653000000.0,4113000000.0,23191000000.0,7134000000.0,-16713000000.0,34584000000.0,65737000000.0,43688000000.0,1770647000.0,4064000000.0,...,4397275.0,-7300967.0,-21138000.0,3930000.0,2986000.0,-3766024.0,-9170000.0,9655494.0,-16647687.0,-34219000.0
EBIT Margin,0.1974,0.2908,0.3273,0.3048,-0.1374,0.379,0.2527,0.3472,0.3068,0.0695,...,0.1905,0.0,0.0,0.1108,0.0,0.0,-16.4928,0.1759,0.0,-0.6454
EBITDA,29329000000.0,6410000000.0,32276000000.0,12558000000.0,-8489000000.0,36647000000.0,78284000000.0,55370000000.0,2071367000.0,4592000000.0,...,6089738.0,-7285140.0,-18705000.0,5710000.0,2986000.0,-3757783.0,-8974000.0,10063235.0,-16441920.0,-30589000.0
EBITDA Margin,0.31,0.453,0.456,0.537,-0.07,0.402,0.301,0.44,0.359,0.079,...,0.264,-0.471,NaN,0.161,NaN,-0.672,-16.14,0.183,NaN,-0.577
EPS,2.56,0.66,4.57,5.67,-2.62,2.64,11.97,5.11,0.26,3.3,...,0.37,-9.6,-0.59,0.06,0.6,-0.08,-0.92,0.52,-0.18,-1.26
EPS Diluted,2.53,0.66,4.48,5.51,-2.62,2.61,11.89,5.06,0.26,3.26,...,0.37,-9.6,-0.59,0.06,0.6,-0.08,-0.92,0.52,-0.18,-1.26
